In [1]:
import os
from dotenv import load_dotenv
from numpy import vectorize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json_decoder as jsd
import plotly.graph_objects as go
import string

s=""
sample_files = []
load_dotenv()

Space_Cleared_Exts = os.environ["EXTENSIONS"].replace(" ", "")
# ------------------------------------------------------------------
# Parcourt tout les fichiers finissant avec les extensions dans .env
# ------------------------------------------------------------------
for ext in os.environ["EXTENSIONS"] :
    if(ext==",") :
        ext="["
        for doc in os.listdir("./FilesToTest/") :
            if doc.endswith(str(s)) :
                sample_files.append(''.join(doc))
        s = ""
    if(ext!="[" and ext!="]") :
        s+=ext
for doc in os.listdir("./FilesToTest/") :
    if doc.endswith(str(s.replace(" ",""))) :
        sample_files.append(''.join(doc))
 
sample_contents = [open("./FilesToTest/"+File).read() for File in sample_files]
print(sample_contents)

['var x = math.floor(math.random() * 5);\nbeed = 2\nvar kek = "kek";\n\nxD LA VIE\n\nif(x==1) {\n    b = 3\n} else {\n    b = 1\n}\n\n\n// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', 'Content3\nbeed = 5\n// Normal Commentary', 'yes\n// Normal Commentary', 'HEHE\n\n// C Normal Commentary \n\n/*\nC High Commentary\n*/']


In [2]:
Commentary_Content = []
Var_Content = []
Equal_Content = []
Loop_Content = []

Sample_Content_Double = []
Sample_Content_Without_CommsLoops = []
Sample_Content_Without_CommsLoopsVar = []
# ---------------------------------------------------------------------------------
# Parcourt notre texte et enleve les commentaires si le texte vient d'un fichier JS
# ---------------------------------------------------------------------------------

def PurgeCommentary(com_format,ext_com_format, syntax) :
    Sample_ContentsBis = sample_contents[syntax].splitlines(True)
    Sample_Content_SubFile = []
    Commentary_SubFile = []
    Long_Comment = False
    Append_Lines = False    
    Stop_Ext_Com = "None"
    
    for lines in Sample_ContentsBis :
        for syntax in com_format :
            for ext in ext_com_format :
                if(Stop_Ext_Com in lines):
                    Long_Comment = False
                    Append_Lines = True
                if(Long_Comment) :
                    Append_Lines = True
                if(syntax in lines) :
                    Append_Lines = True
                if(ext in lines):
                    Append_Lines = True
                    Long_Comment = True
                    if(ext=="/*") :
                        Stop_Ext_Com = "*/"
        if(Append_Lines == False and Stop_Ext_Com not in lines and Long_Comment == False):
            Sample_Content_SubFile.append(lines)
        if(Append_Lines and Long_Comment == False) :
            Commentary_SubFile.append(lines)
            Append_Lines = False
        elif(Append_Lines and Long_Comment) :
            Commentary_SubFile.append(lines)
    Sample_Content_Double.append(Sample_Content_SubFile)
    Commentary_Content.append(Commentary_SubFile)

In [3]:
def PurgeLoops(format, index) :
    Sample_Content_SubFile = []
    Loop_SubFile = []
    IsLoop = False
    Append_Lines = False
    Stop_Loop_Text = "None"
    for lines in Sample_Content_Double[index] :
        for syntax in format :
            if(Stop_Loop_Text in lines) :
                Append_Lines = True
                IsLoop = False
            if(IsLoop) :
                Append_Lines = True
            if(syntax in lines) :
                Append_Lines = True
                IsLoop = True
                if(syntax=="{") :
                    Stop_Loop_Text = "}"
        if(Append_Lines == False and Stop_Loop_Text not in lines and IsLoop == False):
            Sample_Content_SubFile.append(lines)
        if(Append_Lines and IsLoop == False) :
            Loop_SubFile.append(lines)
            Append_Lines = False
        elif(Append_Lines and IsLoop) :
            Loop_SubFile.append(lines)
    Sample_Content_Without_CommsLoops.append(Sample_Content_SubFile)
    Loop_Content.append(Loop_SubFile)

In [4]:
def PurgeVariables(format, index) :
    Sample_Content_SubFile = []
    Var_SubFile = []
    Append_Lines = False
    for lines in Sample_Content_Without_CommsLoops[index] :
        for syntax in format : 
            if(syntax in lines) :
                Append_Lines = True
        if(Append_Lines) :
            Var_SubFile.append(lines)
            Append_Lines = False
        else :
                Sample_Content_SubFile.append(lines)
    Sample_Content_Without_CommsLoopsVar.append(Sample_Content_SubFile)
    Var_Content.append(Var_SubFile)

In [5]:
    for index in range(len(sample_files)) :
        increment_text = False
        temp_text = ""
        for j in range(len(sample_files[index])) :
            if(increment_text) :
                temp_text += (sample_files[index][j]).upper()
            if(sample_files[index][j] == ".") :
                increment_text = True
        format_dict = jsd.decodeFormat(temp_text)

        PurgeCommentary(format_dict['com'],format_dict['extended_com'],index)
        PurgeLoops(format_dict['loop'],index)
        PurgeVariables(format_dict['var'],index)
    print(Sample_Content_Without_CommsLoopsVar)

        

[['\n', 'xD LA VIE\n', '\n', '\n', '\n'], ['Content3\n'], ['yes\n'], ['HEHE\n', '\n', '\n']]


In [6]:
All_Content = []
All_Content_Name = []


print("Commentary Content : \n")
print(Commentary_Content)
print("\nLoop Content : \n")
print(Loop_Content)
print("\nVar Content : \n")
print(Var_Content)

All_Content.append(Commentary_Content)
All_Content.append(Var_Content)
All_Content.append(Loop_Content)

All_Content_Name.append("Commentary")
All_Content_Name.append("Loop")
All_Content_Name.append("Var")



Commentary Content : 

[['// DDDDDDDDDDDDdd\n', '/* HEHEBOY \n', '\n', '\n', '\n', 'SDqsd\n', '*/'], ['// Normal Commentary'], ['// Normal Commentary'], ['// C Normal Commentary \n', '/*\n', 'C High Commentary\n', '*/']]

Loop Content : 

[['if(x==1) {\n', '    b = 3\n', '} else {\n', '    b = 1\n', '}\n'], [], [], []]

Var Content : 

[['var x = math.floor(math.random() * 5);\n', 'beed = 2\n', 'var kek = "kek";\n'], ['beed = 5\n'], [], []]


In [7]:
def tokenize() :
    return ""
tokenize()

''

In [8]:
# vectorize = lambda Text: TfidfVectorizer().fit_transform(Text).toarray()

def vectorize(Text) :
    return TfidfVectorizer(analyzer="char").fit_transform(Text).toarray()

# similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

def similarity(doc1,doc2) :
    return cosine_similarity([doc1,doc2])

In [9]:

def check_plagiarism(TestFiles):
    vectors = vectorize(TestFiles)
    s_vectors = list(zip(sample_files, vectors))
    results = set()
    for sample_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((sample_a, text_vector_a))
        del new_vectors[current_index]
        for sample_b, text_vector_b in new_vectors:
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            sample_pair = sorted((sample_a, sample_b))
            score = sample_pair[0], sample_pair[1], sim_score
            results.add(score)
    return results

In [10]:

import matplotlib.pyplot as plt
from sty import fg

def ExecutePlagiarismChecker(ArrayToTest):
    for array_index in range(len(ArrayToTest)):
        temp_text = ""
        for item_index in range(len(ArrayToTest[array_index])):
            temp_text += ArrayToTest[array_index][item_index]
        ArrayToTest[array_index] = temp_text
    return check_plagiarism(ArrayToTest)
Scoring = 0

for index in range(len(All_Content)) : 
    print(fg.white+"\n"+All_Content_Name[index]+" :")  
    for score in ExecutePlagiarismChecker(All_Content[index]) :
        result = (','.join(map(str,(score)))).split(',')
        if(float(result[2])>=0.6) :
            color = fg.red
        else :
            color = fg.green
        print(color+str(score))
        if(float(result[2])>=0.6) :
            print(fg.white+str(All_Content[index]))



Commentary :
('carabistouille.c', 'testingfile.js', 0.8715073686353488)
['// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', '// Normal Commentary', '// Normal Commentary', '// C Normal Commentary \n/*\nC High Commentary\n*/']
('no.js', 'testingfile.js', 0.08419205479808882)
('carabistouille.c', 'yes.js', 0.8715073686353488)
['// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', '// Normal Commentary', '// Normal Commentary', '// C Normal Commentary \n/*\nC High Commentary\n*/']
('testingfile.js', 'yes.js', 1.0)
['// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', '// Normal Commentary', '// Normal Commentary', '// C Normal Commentary \n/*\nC High Commentary\n*/']
('no.js', 'yes.js', 0.08419205479808882)
('carabistouille.c', 'no.js', 0.15099875184329706)

Loop :
('no.js', 'yes.js', 0.0)
('testingfile.js', 'yes.js', 0.0)
('carabistouille.c', 'testingfile.js', 0.0)
('no.js', 'testingfile.js', 0.5875230680947945)
('carabistouille.c', 'no.js', 0.0)
('carabistouille.c', 'yes.js', 0.0)

Var :
('